In [3]:
import csv
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
#downloaded in MobileHome nltk_data: corpora, taggers (perceptron), tokenizers(punkt)
from nltk.stem.snowball import SnowballStemmer
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer,  TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

import matplotlib
from matplotlib import pyplot as plt


from nltk.tokenize import PunktSentenceTokenizer

from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from nltk import NaiveBayesClassifier


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/MobileHome/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/MobileHome/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/MobileHome/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# https://pythonprogramming.net/new-data-set-training-nltk-tutorial/
# training is done on a movie reviews dataset, provided in the website above
# the movie review dataset is composed of short sentences of lenght comparable to the spooky author dataset

In [4]:
short_pos = open("short_reviews/positive.txt","r", encoding = "ISO-8859-1").read()
short_neg = open("short_reviews/negative.txt","r", encoding = "ISO-8859-1").read()

In [5]:
import random
documents = []

for r in short_pos.split('\n'):
    documents.append( (r, "pos") )

for r in short_neg.split('\n'):
    documents.append( (r, "neg") )

random.shuffle(documents)

In [6]:
#Feature Building Procedure 2
data=pd.DataFrame(documents)
data.columns= ['text', 'review']
data

,text,review
0,whaley's determination to immerse you in sheer...,neg
1,suffers from over-familiarity since hit-hungry...,neg
2,"a distinctly mixed bag , the occasional bursts...",neg
3,"an endlessly fascinating , landmark movie that...",pos
4,"ramsay , as in ratcatcher , remains a filmmake...",pos
...,...,...
10659,at least it's a fairly impressive debut from t...,neg
10660,"a huge box-office hit in korea , shiri is a mu...",pos
10661,"while not all that bad of a movie , it's nowhe...",pos
10662,this is a film living far too much in its own ...,neg


In [7]:
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
stemmer = SnowballStemmer("english")
def clean(doc):
    punc_free = ''.join(ch for ch in doc if ch not in exclude)
    # ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
    # POS_LIST = [NOUN, VERB, ADJ, ADV]
    # lemma.lemmatize("rocks")
    # lemma.lemmatize("better", pos="a")
    normalized_1 = " ".join(lemma.lemmatize(word, 'a') for word in punc_free.split()) 
    normalized_2 = " ".join(lemma.lemmatize(word, 'n') for word in normalized_1.split()) #this deleted s from as
    normalized_3 = " ".join(stemmer.stem(word) for word in normalized_2.split()) #this is very effective
    stop_free = " ".join([i for i in normalized_3.lower().split() if i not in stop])
    return stop_free

In [8]:
data['text_clean'] = data['text'].apply(clean) #clean(data['text'][0]) 

In [8]:
def cv(data):
    """
    Count vectorizing function.  Returns embedded vector and vectorizer.
    """
    count_vectorizer = CountVectorizer() # Bag-of-words vectorization
#     count_vectorizer = TfidfVectorizer() # Term Frequency-Inverse Document Frequency vectorization
    #count_vectorizer = CountVectorizer(ngram_range =(1, 2)) # n-gram vectorization  
    emb = count_vectorizer.fit_transform(data)

    return emb, count_vectorizer

In [9]:
emb, dictionary = cv(data['text_clean'].tolist())
emb.todense().shape

(10664, 13927)

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

def gen_Classifier(data, emb, classifier):     
    # freq= pd.Series((' '.join(data['text']).split())).valuecounts().sortvalues(ascending = False)
    # vocab_ = barplotTopWords(freq)
    array = emb.todense()
    emb2= pd.DataFrame(array)
    emb2['output'] = data['review']
    

    features = emb2.columns.tolist()
    output = 'output'
    # removing the output and the id from features
    features.remove(output)
    
    if classifier == RandomForestClassifier:
        list1 = [False, True]
        parameter_grid =[{"bootstrap": list1 }]
        score= 'accuracy'
    elif classifier == MultinomialNB:
        list1 = np.linspace(0.006, 0.1, 2)
        list1 = np.around(list1, decimals=4)
        parameter_grid =[{"alpha": list1}]
        score= 'accuracy'
        
    classifier1= classifier()
    gridsearch1 = GridSearchCV(classifier1,parameter_grid, scoring = score, cv = 2)
    
    my_model= gridsearch1.fit(emb2[features], emb2[output])
    return my_model

gridsearch1 = gen_Classifier(data, emb, RandomForestClassifier)

/Users/MobileHome/anaconda3/envs/tf2/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/MobileHome/anaconda3/envs/tf2/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/MobileHome/anaconda3/envs/tf2/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/MobileHome/anaconda3/envs/tf2/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", Fut

In [11]:
gridsearch1.estimator

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators='warn',
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [65]:
# training_set = pd.DataFrame (emb.todense() [:round(emb.shape[0]/2)])
# testing_set = pd.DataFrame (emb.todense() [round(emb.shape[0]/2):])

array = emb.todense()
emb2= pd.DataFrame(array)
emb2['output'] = data['review']
    

features = emb2.columns.tolist()
output = 'output'
# removing the output and the id from features
features.remove(output)

emb2.head()

# vectorizer.get_feature_names()

,0,1,2,3,4,5,6,7,8,9,...,13918,13919,13920,13921,13922,13923,13924,13925,13926,output
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,neg
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,pos
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,neg
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,pos
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,neg


In [ ]:
training_set = emb2 [:round(emb.shape[0]/2)]
testing_set = emb2 [round(emb.shape[0]/2):]

In [43]:
def predict_Classifier(testing_set, model):
    prediction= model.predict(testing_set)
#     if model.estimator == RandomForestClassifier:
#         results2= model.best_estimator_.feature_importances_
#     results2=(nltk.classify.accuracy(model, testing_set))*100
#     elif model.estimator == MultinomialNB:
#     results2= model.best_params_
        
    return prediction, results2, model.best_params_, model.best_score_ , model.best_estimator_  




In [67]:
prediction, results2, best_params, best_score, best_estimator= predict_Classifier(testing_set[features], gridsearch1)
print(results2, best_params, best_score, best_estimator)

[] {'bootstrap': False} 0.6927044261065266 RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


In [68]:
n_correct= sum(prediction == data['review'][round(emb.shape[0]/2):])
me= n_correct/len(prediction)
me
#this is so high because I am testing on the same dataset as the training set, 
#should I only consider the value from the GRIDSearch

0.9998124531132783

In [69]:
featuresimportance= gridsearch1.best_estimator_.feature_importances_

In [70]:
dictnames= dictionary.get_feature_names()
len(dictnames)

13927

In [71]:
featureimportancewithnames = pd.DataFrame(np.column_stack([dictnames, featuresimportance]), 
                               columns=['name', 'importance'])


In [72]:
Mostimp= featureimportancewithnames.sort_values( ['importance'], axis=0, ascending=False, inplace=False, kind='quicksort')
Mostimp.head()

,name,importance
2559,compress,9.999802672255283e-07
6173,industri,9.986964826582374e-07
11191,sloppi,9.985509633278748e-05
11876,suicid,9.976125352183732e-06
2400,coal,9.974601983605891e-07


In [73]:
import pickle

save_classifier = open("pickled/SentimentAnalysisTrainClassifierMine.pickle","wb")
pickle.dump(gridsearch1, save_classifier)
save_classifier.close()

In [74]:
import pickle
classifier_f = open("pickled/SentimentAnalysisTrainClassifierMine.pickle", "rb")
classifier = pickle.load(classifier_f)
classifier_f.close()

In [82]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

from nltk.classify import ClassifierI
from statistics import mode

from sklearn.metrics import accuracy_score


In [76]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [57]:
training_set[features]

,0,1,2,3,4,5,6,7,8,9,...,13917,13918,13919,13920,13921,13922,13923,13924,13925,13926
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5327,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5328,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5329,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5330,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [95]:
MNB_classifier = MultinomialNB()
MNB_classifier.fit(training_set[features], training_set[output])
predicted = MNB_classifier.predict(testing_set[features])
print (accuracy_score(testing_set[output], predicted))

BernoulliNB_classifier = BernoulliNB()
BernoulliNB_classifier.fit(training_set[features], training_set[output])
predicted = classifier1.predict(testing_set[features])
print (accuracy_score(testing_set[output], predicted))

# LogisticRegression_classifier = LogisticRegression()
# LogisticRegression_classifier.fit(training_set[features], training_set[output])
# predicted = classifier1.predict(testing_set[features])
# print (accuracy_score(testing_set[output], predicted))

# SGDClassifier_classifier = SGDClassifier()
# SGDClassifier_classifier.fit(training_set[features], training_set[output])
# predicted = classifier1.predict(testing_set[features])
# print (accuracy_score(testing_set[output], predicted))

##SVC_classifier = SVC()
##SVC_classifier.fit(training_set[features], training_set[output])
# predicted = classifier1.predict(testing_set[features])
# print (accuracy_score(testing_set[output], predicted))

# LinearSVC_classifier = LinearSVC()
# LinearSVC_classifier.fit(training_set[features], training_set[output])
# predicted = classifier1.predict(testing_set[features])
# print (accuracy_score(testing_set[output], predicted))

# NuSVC_classifier = NuSVC()
# NuSVC_classifier.fit(training_set[features], training_set[output])
# predicted = classifier1.predict(testing_set[features])
# print (accuracy_score(testing_set[output], predicted))



0.7554388597149287
0.8998499624906227


In [87]:
voted_classifier = VoteClassifier(classifier,
#                                   NuSVC_classifier,
#                                   LinearSVC_classifier,
#                                   SGDClassifier_classifier,
#                                   LogisticRegression_classifier,
                                  BernoulliNB_classifier,
                                  MNB_classifier)

print("voted_classifier accuracy percent:", voted_classifier.__class__.__name__, accuracy_score(testing_set[output], predicted))

voted_classifier.predict()

voted_classifier accuracy percent: VoteClassifier 0.8998499624906227


In [ ]:
save_classifier = open("pickled/SentimentAnalysisTrainMNB_classifier.pickle","wb")
pickle.dump(MNB_classifier, save_classifier)
save_classifier.close()

save_classifier = open("pickled/SentimentAnalysisTrainBernoulliNB_classifier.pickle","wb")
pickle.dump(BernoulliNB_classifier, save_classifier)
save_classifier.close()

In [ ]:
# #NOT USED
# all_words = []

# short_pos_words = nltk.word_tokenize(short_pos)
# short_neg_words = nltk.word_tokenize(short_neg)

# for w in short_pos_words:
#     all_words.append(w.lower())

# for w in short_neg_words:
#     all_words.append(w.lower())

# all_words = nltk.FreqDist(all_words)
# all_words
# len(all_words)

In [ ]:
# # move this up here
# all_words = []
# documents = []


# #  j is adject, r is adverb, and v is verb
# #allowed_word_types = ["J","R","V"]
# allowed_word_types = ["J"]

# for p in short_pos.split('\n'):
#     documents.append( (p, "pos") )
#     words = word_tokenize(p)
#     pos = nltk.pos_tag(words)
#     for w in pos:
#         if w[1][0] in allowed_word_types:
#             all_words.append(w[0].lower())


# for p in short_neg.split('\n'):
#     documents.append( (p, "neg") )
#     words = word_tokenize(p)
#     pos = nltk.pos_tag(words)
#     for w in pos:
#         if w[1][0] in allowed_word_types:
#             all_words.append(w[0].lower())

# all_words = nltk.FreqDist(all_words)

# word_features = list(all_words.keys())[:5000]

# def find_features(document):
#     words = word_tokenize(document)
#     features = {}
#     for w in word_features:
#         features[w] = (w in words)

#     return features

# featuresets = [(find_features(rev), category) for (rev, category) in documents]


In [ ]:
from sklearn.ensemble import VotingClassifier
a= MultinomialNB()
b= BernoulliNB()
voting= VotingClassifier(estimators=[('M', a), ('b', b)], voting= 'hard')

voting.fit(testing_set[features], testing_set[output])

In [ ]:
for clf in (a, b, voting):
    clf.fit(testing_set[features], testing_set[output])
    y_pred= clf.predict(testing_set[features])
    print(clf.__class__.__name__, accuracy_score(testing_set[output], y_pred))